In [1]:
import pandas as pd
import os

In [3]:
file_location = '/home/jasonmellone/Documents/nyu/DSQF/analyst_forecast/'

In [17]:
data_files = list(filter(lambda x: 'csv' in x, os.listdir(file_location)))

In [18]:
data_files_full_path = list(map(lambda x: file_location + x,data_files))

In [19]:
data = dict(zip(data_files,map(pd.read_csv,data_files_full_path)))

KeyboardInterrupt: 

In [20]:
# data

In [21]:
sp500 = pd.read_csv(file_location + 'AFE_data/data/clean_cons_70_17.csv')

   Unnamed: 0  gvkey  gvkeyx      from        thru              conm    tic  \
0         374  10507       3  19640331  20071009.0  S&P 500 Comp-Ltd  I0003   
1        1253  12144       3  20020722  20050811.0  S&P 500 Comp-Ltd  I0003   
2         296   2316       3  19640331  19941220.0  S&P 500 Comp-Ltd  I0003   
3        1147  30095       3  19991116  20030925.0  S&P 500 Comp-Ltd  I0003   
4         484   7148       3  19681231  19750831.0  S&P 500 Comp-Ltd  I0003   

                        co_conm co_tic   co_cusip  
0   ENERGY FUTURE HOLDINGS CORP  0033A  873168108  
1      SUNGARD DATA SYSTEMS INC  0139A  867363103  
2                    HEXION INC  0141A  428300107  
3  QUINTILES TRANSNATIONAL CORP  0573B  748767100  
4                  MCCRORY CORP  1219A  579865007  


<h2>Read in S&P constituent returns, and remove dates not existing in SPX index returns</h2>

In [ ]:
spx_returns = pd.read_csv(file_location + 'SPX.csv',index_col=0)

In [8]:
sp500_constituent_returns = None
sp500_constituent_returns = pd.read_csv(file_location +'AFE_data/data/crsp_sp500_1970_2016.csv')
sp500_constituent_returns.date = pd.to_datetime(sp500_constituent_returns.date, format='%Y%m%d')
sp500_constituent_returns = sp500_constituent_returns[sp500_constituent_returns.date.isin(spx_returns.index)]

sp500_constituent_returns.to_pickle(file_location +'AFE_data/data/crsp_sp500_1970_2016_dates_cleaned.p')